# RAG pipeline using Langchain, RAPTOR, and MilvusDB
This notebook uncovers the state-of-the-art techniques in the Retrieval Augmented Generation (RAG) models built for tasks such as Question-Answering (QA), text generation and summarization from context, and much more.
It uses Langchain as the backbone and Milvus db for storing vector embeddings of the text along with RAPTOR for optimized results (since it captures low-level as well as high-level meanings from the text).

#### Loading the documents

In the root directory of the project, there is a `data/` directory which stores all the pdfs. The function below loads all the pdfs one by one, ignores the irrelevant pages (in terms of value added to the context) from the beginning and end of the book, and finally formats the page content by correcting multiple spaces and newline characters.

In [1]:
import os
import re
import warnings
from langchain.document_loaders import PyPDFLoader

warnings.filterwarnings('ignore')

def select_relevant_pages_and_format_text(directory: str) -> list:
    """
    Selects relevant pages from all pdfs in a directory and formats their text

    Input:
    - directory: path to the folder with all pdfs

    Output:
    - list of documents/all pages from all pdfs
    """
    relevant_pages = []

    for i in os.listdir(directory):
        path = os.path.join(directory, i)
        loader = PyPDFLoader(path)
        print(f"[INFO] Loading pages from {i}...")
        if i == 'book1.pdf':
            relevant_pages.extend(loader.load()[8:328])
        elif i == 'book2.pdf':
            relevant_pages.extend(loader.load()[13:289])
        elif i == 'book3.pdf':
            relevant_pages.extend(loader.load()[15:300])

    print(f"[INFO] Formatting text...")
    for i in relevant_pages:
        # delete blank/sparse pages
        if len(i.page_content) <= 100:
            idx = relevant_pages.index(i)
            relevant_pages.pop(idx)
        # remove multiple spaces and newline chars
        i.page_content = i.page_content.replace('\n', ' ')
        i.page_content = re.sub(r'\s{2,}', ' ', i.page_content)

    print(f"[INFO] Loaded {len(relevant_pages)} pages from all pdfs.")
    
    return relevant_pages

In [2]:
relevant_pages = select_relevant_pages_and_format_text('data')

[INFO] Loading pages from book2.pdf...
[INFO] Loading pages from book3.pdf...
[INFO] Loading pages from book1.pdf...
[INFO] Formatting text...
[INFO] Loaded 709 pages from all pdfs.


#### Splitting
Once relevant pages have been extracted from the pdfs, and their text has been formatted, we split the context into tokens (~100 each, keeping a 10% overlap).
According to this code, **709 pages have been split into 3150 chunks of approx 100 tokens each**

In [3]:
from langchain_text_splitters import TokenTextSplitter

print(f"[INFO] Splitting into chunks...")

text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = text_splitter.split_documents(relevant_pages)

print(f"[INFO] {len(chunks)} chunks created.")

[INFO] Splitting into chunks...
[INFO] 3150 chunks created.


In [4]:
import random

rand_idx = random.randint(0, len(relevant_pages))

print(f"[INFO] No of original pages:     {len(relevant_pages)}")
print(f"[INFO] No of chunks generated:   {len(chunks)}")
# since 1 token = approx. 4 chars
print(f"[INFO] Tokens in original text:  {len(relevant_pages[rand_idx].page_content) // 4}")
print(f"[INFO] Tokens in chunked text:   {len(chunks[rand_idx].page_content) // 4}")

[INFO] No of original pages:     709
[INFO] No of chunks generated:   3150
[INFO] Tokens in original text:  716
[INFO] Tokens in chunked text:   125


#### Initializing the LLM
This project uses Llama.cpp for running a LLM locally and there is a **.gguf** model file saved locally in the project directory. Alternate options include running any of the LLMs mentioned here: https://python.langchain.com/v0.2/docs/integrations/llms/

In [5]:
from langchain_community.llms import LlamaCpp

model_path = "zephyr-quiklang-3b-4k.Q4_K_M.gguf"

print(f"[INFO] Loading {model_path}...")
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.1,
    f16_kv=True,
    n_ctx=2048,
    verbose=False
)

print(f"[INFO] Loaded model successfully")

[INFO] Loading zephyr-quiklang-3b-4k.Q4_K_M.gguf...
[INFO] Loaded model successfully


#### Initializing a vector database
This project uses Milvus db for storing the vector embeddings but any one of ChromaDB, Pinecone, FAISS, etc can be used (see respective documentation).
Along with this, for embedding model `all-MiniLM-L6-v2` has been used since its performance is pretty good given that its very lightweight.\
See other models here: https://sbert.net/docs/sentence_transformer/pretrained_models.html \
\
*Note: I have edited the `milvus.py` file locally to correctly integrate the sentence-transformer embedding model. Make sure you edit the error generating file locally, and restart the jupyterlab/notebook* 

In [6]:
from sentence_transformers import SentenceTransformer

model_name = "all-MiniLM-L6-v2"
print(f"[INFO] Loading {model_name}...")
embed_model = SentenceTransformer(model_name)
print(f"[INFO] Loaded model successfully")

[INFO] Loading all-MiniLM-L6-v2...
[INFO] Loaded model successfully


In [7]:
import math
from langchain_milvus.vectorstores import Milvus

print(f"[INFO] Creating vector database...")

try:
    del vectorstore
except:
    pass

vectorstore = Milvus.from_documents(
    documents=chunks,
    embedding=embed_model,
    connection_args={"uri": "./first.db",},
    drop_old=True,
)

print(f"[INFO] Created milvus collection from {len(chunks)} docs")

[INFO] Creating vector database...
[INFO] Created milvus collection from 3150 docs


#### Hybrid retrieval
I have created a list of queries going through the texts and will use any random one each time while running the RAG chain. Also, I am selecting the top 50 chunks similar to the given query in terms of cosine distance of their embeddings from the vectordb as well as top 50 from the **BM25 (best matching algo) retriever**.\
Finally, an ensemble retriever selects the most relevant documents which are then fed into the RAPTOR tree generating function as leaves.

In [8]:
queries = ["How do bureaucracy and corruption impact foreign investments?",
           "Describe Proactive Development and Intelligence.",
           "What are different threat levels?",
           "Why does Saudi hold such importance in global economy?",
           "How would you optimize decision trees?",
           "What is the function decomposition in machine learning?",
           "Briefly define kernel-based machine translation.",
           "What are different approaches to Dedicated Word Selection?",
           "What was the ACAI 1999 workshop on ML in Human Language Technology?",
           "Mention some genetic algorithms from the book.",
           "Describe the Arachne System for noise reduction."]

In [9]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 50

milvus_retriever = vectorstore.as_retriever(search_kwargs={'k': 50})

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, milvus_retriever],
                                       weights=[0.4, 0.6])

query = random.choice(queries)

# Retrieve relevant documents
similar_chunks = ensemble_retriever.get_relevant_documents(query)

#### RAPTOR Implementation
The following implementation is based on the RAPTOR tree clustering methodology to extract both high and low level details from the text to generate a more comprehensive response which is qualitatively better and more accurate/relevant to the given context.

This code has been taken from the following repos. Full credit to all the authors:

-   [Original paper](https://arxiv.org/pdf/2401.18059), [repo](https://github.com/parthsarthi03/raptor/blob/master/raptor/cluster_tree_builder.py)
-   [Minor tweaks](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-raptor/llama_index/packs/raptor/clustering.py)
-   [YT video repo](https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb)

Apart from the standard implementaion, some minor changes have been made to suit the embedding model, LLM, and hardware resources.

In [10]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap.umap_ as umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 224  # Fixed seed for reproducibility

### --- Code from citations referenced above (added comments and docstrings) --- ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


### --- Our code below --- ###


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embed_model.encode(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"[INFO] Generated {len(all_clusters)} cluster(s)")

    # Summarization
    template = """Here is a collection of books on Machine Learning, AI, and related topics. 
    Give a detailed summary of the documentation provided.
    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

In [11]:
# Build tree
leaf_texts = [c.page_content for c in similar_chunks]
results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

[INFO] Generated 21 cluster(s)
[INFO] Generated 4 cluster(s)
[INFO] Generated 1 cluster(s)


#### Reranking context
Now that the tree has been generated, we combine all the leaf nodes and their cluster summaries into one big list of text and rerank those according to relevance to original query. Finally, a basic rag chain takes the content of the vectorstore as input and generates an response accordingly.

In [12]:
all_texts = leaf_texts.copy()

# iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    summaries = results[level][1]["summaries"].tolist()
    all_texts.extend(summaries)

In [13]:
from sentence_transformers import CrossEncoder

# reranking the selected queries
model = CrossEncoder("mixedbread-ai/mxbai-rerank-base-v1")
rerank = model.rank(query, all_texts, return_documents=True, top_k=3)
reranked_text = [r['text'] for r in rerank]

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_milvus.vectorstores import Milvus

try:
    del ranked_vectorstore
except:
    pass

print(f'[INFO] Creating a vectorstore from reranked context...')
ranked_vectorstore = Milvus.from_texts(
    texts=reranked_text,
    embedding=embed_model,
    connection_args={"uri": "./second.db",},
    drop_old=True,
)
print(f'[INFO] Vectorstore created.')

template = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and based on context as far as possible.

Assistant:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

retriever = ranked_vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

[INFO] Creating a vectorstore from reranked context...
[INFO] Vectorstore created.


In [15]:
print(f"[QUESTION]: {query}")
output = rag_chain.invoke(query)
print(f"[RESPONSE]: {output.strip()}")

[QUESTION]: Describe Proactive Development and Intelligence.
[RESPONSE]: Proactive Development and Intelligence (PDI), as mentioned in the context, focuses on identifying opportunities and generating future profit by carefully analyzing market intelligence and reality audits. It involves various aspects such as continuous collection and processing of security intelligence, discreet security appraisals, threat lead assessments, systematic external inspections, security education and training, instructions/regulations, physical security, personnel security, drills and procedures, and protective security. The main goal of PDI is to ensure a regular and accurate flow of information and make proactive preparations to deal with potential crises or threats in Machine Learning, AI, and other related fields.
